In [3]:
import torch
import torch, torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models
import os, numpy as np
from torch.utils.data import Dataset, DataLoader

In [4]:
# set device

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
# Customized dataset

from torch.utils.data import Dataset
class SimpleDataset(Dataset):
    def __init__(self, x, y, transform=None):
        self.data = x
        self.targets = y
        self.transform = transform

    def __getitem__(self, index):
        img = self.data[index]
        if self.transform is not None:
            img = self.transform(img)
        target = self.targets[index]
        return img, target

    def __len__(self):
        return len(self.data)

In [ ]:
# Data transforms (Cifar10)+ Dataloaders
data_transforms = {
     'train': transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
}

trainset = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=data_transforms['train'])
train_loader = torch.utils.data.DataLoader(trainset, batch_size=256,
                                          shuffle=True, num_workers=10)

validset = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=data_transforms['val'])
valid_loader = torch.utils.data.DataLoader(validset, batch_size=256,
                                         shuffle=False, num_workers=10)

In [ ]:
# Torch save model and load model

model = models.resnet18()
torch.save(model.state_dict(), PATH)

model = models.resnet18()
model.load_state_dict(torch.load(PATH))
model.eval()

In [ ]:
# Print model parameters

for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data, param.data.shape)

In [ ]:
# Parallel Training

model = models.resnet18()
model = nn.DataParallel(model).cuda()

In [ ]:
import torch, torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.models as models
import matplotlib.pyplot as plt
import time, os, copy, numpy as np
from livelossplot import PlotLosses
import sys

# Learning Rate
optimizer_ft = optim.SGD(model.parameters(), lr=1e-2, momentum=1e-2)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=100, gamma=0.1) 

In [ ]:
dataloaders = {'train':train_loader, 'val': valid_loader}
dataset_sizes = {'train':len(trainset.data), 'val': len(validset.data)}

In [ ]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, num_epochs=25, model_path = ''):
    device = 'cuda'
    since = time.time()
    liveloss = PlotLosses()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        # scheduler.step()
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for i,(inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                print("Epoch[{}/{}]({}/{}): Loss: {:.4f}".format(epoch+1, num_epochs, i, \
                                                                 len(dataloaders[phase],loss.item() * inputs.size(0))))


                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            if phase == 'train':
                avg_loss = epoch_loss
                t_acc = epoch_acc
            else:
                val_loss = epoch_loss
                val_acc = epoch_acc
            

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                best_epoch = epoch
                torch.save(model.state_dict(), model_path)
                
                
        liveloss.update({
            'log loss': avg_loss,
            'val_log loss': val_loss,
            'accuracy': t_acc,
            'val_accuracy': val_acc
        })
                
        liveloss.draw()
        print('Train Loss: {:.4f} Acc: {:.4f}'.format(avg_loss, t_acc))
        print( 'Val Loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc))
        print('Best Val Accuracy: {}'.format(best_acc))
        print('Best epoch: {}'.format(epoch))
        print()
        
        if epoch-best_epoch > 20:
            print('model stopped due to early stopping')
            break
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model